# Swarm Simulator for Outer Space

This swarm simulator will explore different swarm algorithms for an application in outer space. This approach will divide a plot into grid squares, which can then be analysed to determine and control the behaviour of each individual agent in the swarm.

In [1]:
#imports
import sympy as sym
import plotly as py
import plotly.tools as tls
from plotly import graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots
from IPython.display import Image

import dash
from dash.dependencies import Output, Input
import dash_core_components as dcc
import dash_html_components as html

import numpy as np
import pandas as pd
import ipywidgets as widgets
import time
import math

from scipy import special
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from collections import deque
from scipy.spatial.distance import cdist, pdist, euclidean
from sympy import sin, cos, symbols, lambdify

from robot import Robot
from dust_devil import DustDevil

import random
import datetime
import os
import math

import Processing_Functions

app = dash.Dash(__name__)

In [2]:
#Plotly offline mode
init_notebook_mode(connected=True)

## Initialisation of Constants

In [3]:
#initialising parameters
experiment = False
conversion_ms = 1/3.6 #km/h to m/s conversion actor
timer = 0


#Robot Parameters
robot_number = 200
robot_speed = 13*conversion_ms #m/s
robot_radius = 50 
robot_mass = 20 #kg
x_robot = []
y_robot = []
min_neighbour = []
gen_random = 0 #the random angle

#dust devil parameters
dust_time = 90 #time in seconds the dust devil will survive 
dust_speed = 150*conversion_ms #m/s
dust_radius = 100
dust_timer_count = 0
dust_devils = []

#Area Parameters
area = 1 #area in km^2 
side = 1 #each side length in m
per_square_km = 100/(60*60*24)
probability_dust = (per_square_km*area) #number of dust devils per second

#Sensor Parameters
sensor_radius = 40000

#Mean Squared Distance
MSD = []

## Physics Constants

In [4]:
#setting physics constants
power = 2
R = 200
max_force = 1
multiply = 1.5

In [5]:
#Function to initialise robots
def initialise(robot_number,length):
    '''
    Returns the list of robot objects

            Parameters:
                    robot_number (int): Number of robots
                    length (int): Length of one side of the area

            Returns:
                   robots (list): List of robot objects
    '''
    robots = []
    X,Y = random_position(robot_number,-20,20)
    cnt = 0
    for x,y in zip(X,Y):
        walk_time = int(np.random.normal(5,3,1))
        heading = random.uniform(0,2*math.pi)
        if(cnt == 0):
            identifier = 0
        else:
            identifier = 1
            cnt = -1
        robots.append(Robot(x,y,1,0,0,robot_speed, walk_time,heading,identifier))
        cnt = cnt+1
    return robots

In [6]:
#Function to return randomised x,y position array
def random_position(robot_number,start,finish):
    '''
    Returns random position array of x and y
Failure: Directory creation of Experiments/G_Transition/ failed
Failure: Directory creation of Experiments/G_Transition/ failed
Failure: Directory creation of Experiments/G_Transition/10 Robots/ failed
Failure: Directory creation of Experiments/G_Transition/10 Robots/Images/ failed

            Parameters:
                    robot_number (int): Number of robots
                    length (int): Length of one side of the area

            Returns:
                   x (integer list): List of random x positions
                   y (integer list): List of random y positions
    '''
    
    x = []
    y = []
    for i in range(int(robot_number)):
        x.append(random.randint(start,finish))
        y.append(random.randint(start,finish))
        walk_time = int(np.random.normal(20,5,1))
    return x,y

In [7]:
#Function to return square grid of x,y positions
def grid_center(length):
    '''
    Returns square grid position array of x and y

            Parameters:
                    length (int): Length of one side of the area

            Returns:
                   x (integer list): List of x positions in the grid
                   y (integer list): List of y positions in the grid
    '''
    division = length/6
    center_x = []
    center_y = []
    for i in range(1,6):
        for j in range(1,6):
            center_x.append(division*j)
            center_y.append(division*i)
    return center_x,center_y

In [8]:
#Function to retrieve class object positions
def positions(objects):
    '''
    Returns 2D positions of class objects

            Parameters:
                    objects (list): List of objects

            Returns:
                   x (numpy array int): List of x positions of the object
                   y (numpy array int): List of y positions of the object
    '''
    x = []
    y = []
    for i in range(len(objects)):
       x.append(objects[i].x)
       y.append(objects[i].y) 
    return np.array(x),np.array(y)

In [9]:
#Function to check if the robot has gone over the edge and changing the value to negative if it has
def bounce(x_updated,y_updated,x_change,y_change,length):
    '''
    Returns bounced x and y position

            Parameters:
                    x_updated (float): X position after movement update
                    y_updated (float): Y position after movement update
                    x_change (float):  Change from previous X position
                    y_change (float):  Change from previous Y position
                    length (float):    Length of one side of the area

            Returns:
                   x_updated (float): Bounced X position
                   y_updated (float): Bounced Y position
    '''
    #if the x is larger than the side limit, then bounce off
    if(x_updated > length or x_updated < 0):
        x_updated = x_updated-2*x_change
    #if the y is larger than the side limit, then bounce off
    if(y_updated > length or y_updated < 0):
        y_updated = y_updated-2*y_change
    return x_updated,y_updated

In [10]:
#Function check if there is any collisions between the robots (placeholder)
def collision_detection(swarm,dust_devils):
    positions_robot = np.array([positions(swarm)])
    position_dust = np.array([positions(dust_devils)])
    
    check = False
    return check

In [11]:
#Function to work out the magnitude of a 2D vector
def magnitude(x,y):
    '''
    Finds the magnitude of 2 floats
    
            Parameters:
                   x (float): a float value
                   y (float): a float value
            
            Returns:
                   mag (np array): the resulting np array of magnitudes from the two given values

    '''
    mag = np.sqrt(np.square(x)+np.square(y))
    return mag

In [12]:
#Function to work out a 2D unit vector
def unit(vector):
    '''
    Finds the unit vector of a passed vector
    
            Parameters:
                   Vector(2D numpy array): the 2D position vector
            
            Returns:
                   Unit(2D numpy array): the resulting unit vector from the given vector

    '''
    #print("Vector:", vector)
    mag = magnitude(vector[0],vector[1])
    #print("Magnitude", mag)
    
    if(vector[0] == 0):
        x = 0
    else:
        x = vector[0]/mag
    if(vector[1]==0):
        y = 0
    else:
        y = vector[1]/mag
    unit = np.array([x,y])


    
    return unit
    

In [13]:
#Function to check if division by denominator is non zero, if it is zero, then the component is returned as zero
def division_check(numerator, denominator):
    '''
    Checks if the division is possible without zero division, if not set the component as zero
    
            Parameters:
                   Numerator (float): the numerator of the division
                 Denominator (float): the denominator of the division
            
            Returns:
                     Result (float): the result of the division

    '''

    if(denominator == 0):
        result = 0
    else:
        result = numerator/denominator
    
    return result


In [14]:
#Function to call physics based area coverage algorithm
def physics_walk_honey(swarm,G,power,R,max_force,multiply,lattice,min_neighbours):
    '''
    Updates the robot position based on a physicomimetics algorithm

            Parameters:
                   Swarm (list): a list of robot objects
            
            Returns:
                   None

    '''


    #print("Physics Called \n===============")
    #setting initial constants
    timestep = 1
    force_x = 0
    force_y = 0 
    velocity_x = 0
    velocity_y = 0
    minimum_distance = 0
    amount = len(swarm)
    
    #retrieving x and y positions
    x,y = positions(swarm)


    
    #looping through each robot in the swarm
    
    for robot in swarm:
        
        
        #initialising the forces as 0
        force = np.array([0,0])
        
        R_0 = R
        R_1 = R
        #x_dist_unsorted,y_dist_unsorted = dist(robot,swarm)
        
        #finding the distance between the current robot and the other robots
        x_0,y_0 = dist(robot,x,y)
        x_1,y_1 = dist(robot,x,y)
        #calculating the distance from the robots to the current robot, split up according to robot type
        distance_0 = np.sqrt(np.square(x_0)+np.square(y_0))
        distance_1 = np.sqrt(np.square(x_1)+np.square(y_1))
        
        #finding the minimum of both arrays, to find the minimum distance to neighbours performance parameter
        min_0 = min(distance_0[np.nonzero(distance_0)])
        min_1 = min(distance_1[np.nonzero(distance_1)])
        
        if(min_0<min_1):
            minimum_distance = minimum_distance + min_0
        elif(min_0>min_1):
            minimum_distance = minimum_distance + min_1
        else:
            minimum_distance = minimum_distance + min_0
        
        multiply_like = 1.3
        multiply_unlike = 1.7
        constant = 0
        if(robot.identifier == 0):
            R_0 = lattice*R
            R_1 = R
            distance_local_0 = (distance_0<multiply_like*R)
            distance_local_1 = (distance_1<multiply_unlike*R)
            constant_0 = 1/lattice
            constant_1 = 1
            
        if(robot.identifier == 1):
            R_0 = R
            R_1 = lattice*R
            distance_local_0 = (distance_0<multiply_unlike*R)
            distance_local_1 = (distance_1<multiply_like*R)
            constant_0 = 1
            constant_1 = 1/lattice
            
        
        x_dist_0 = x_0[distance_local_0]
        y_dist_0 = y_0[distance_local_0]    
        position_0 = np.array([x_dist_0,y_dist_0])
        x_dist_1 = x_1[distance_local_1]
        y_dist_1 = y_1[distance_local_1]
        position_1 = np.array([x_dist_1,y_dist_1])
        
        force_0 = artificial_gravity(robot,len(x_dist_0),position_0,max_force,R_0,G,constant_0)
        force_1 = artificial_gravity(robot,len(x_dist_1),position_1,max_force,R_1,G,constant_1)
        
        friction = random.uniform(-0.90,0.9)
        
        force = force_0 + force_1 
        unit_force = unit(force)
        
        avg_min_distance = minimum_distance/len(swarm)
        
        #calculating the change in velocity
        delta_vx = force[0]*timestep/robot.mass
        delta_vy = force[1]*timestep/robot.mass

        #calculating the new velocity
        velocity_x = robot.x_velocity + delta_vx
        velocity_y = robot.y_velocity + delta_vy
        

        
        #keeping the velocity within the maximum velocity of the robot
        if(velocity_x>robot.max_velocity):
            velocity_x = robot.max_velocity*unit_force[0]
        if(velocity_y>robot.max_velocity):
            velocity_y = robot.max_velocity*unit_force[1]
        if(velocity_x<-robot.max_velocity):
            velocity_x = robot.max_velocity*unit_force[0]
        if(velocity_y<-robot.max_velocity):
            velocity_y = robot.max_velocity*unit_force[1]
            
        #updating the robots velocity
        robot.update_velocity(velocity_x,velocity_y)    

        #calculating the change in position for x and y
        x_change = velocity_x*timestep
        y_change = velocity_y*timestep

        #calculating the new position
        x_updated = robot.x+x_change
        y_updated = robot.y+y_change
        
        #updating the position
        robot.update_position(x_updated,y_updated)

    min_neighbours.append(avg_min_distance)
            

In [15]:
#Function to call physics based area coverage algorithm
def physics_walk(swarm,G,power,R,max_force,multiply,min_neighbours,cluster_average):
    '''
    Updates the robot position based on a physicomimetics artificial gravity algorithm

            Parameters:
                   Swarm (list): a list of robot objects
                   G (float): the chosen G constant float value
                   power (float): the chosen power for the artificial gravity equation
                   R (float): the chosen R seperation float value
                   max_force (float): the maximum float limit on the force
                   multiply (float): the range multiplication float
                   min_neighbours (float): the minimum neighbour distance array
                   cluster_average (float): the cluster size average array
            Returns:
                   None

    '''

    #print("Physics Called \n===============")
    #setting initial constants
    timestep = 1
    force_x = 0
    force_y = 0 
    velocity_x = 0
    velocity_y = 0
    minimum_distance = 0
    amount = len(swarm)
    
    #retrieving x and y positions
    x,y = positions(swarm)
    count = 0
    
    
    cluster_total = 0
    
    #looping through each robot in the swarm
    
    for robot in swarm:
        cluster_count = 1
        
        #initialising the forces as 0
        force = np.array([0,0])
        
        #finding the distance between the current robot and the other robots
        x_dist_unsorted = x-robot.x
        y_dist_unsorted = y-robot.y
        
        #calculating the distance from the robots to the current robot
        distance = np.sqrt(np.square(x_dist_unsorted)+np.square(y_dist_unsorted))
        
        #calculating the minimum stance
        minimum_distance = minimum_distance + min(distance[np.nonzero(distance)])
 


        #determining the robots within 1.5R distance
        distance_local = (distance<multiply*R)
        

        #creating the x and y distance matrices for robots within the local 1.5R distance
        x_dist = x_dist_unsorted[distance_local]
        y_dist = y_dist_unsorted[distance_local]
        position = np.array([x_dist,y_dist])
        force_change_dir = 0
        
        robot_number = len(distance_local)
        
        
        
        
        for i in range(len(x_dist)):
            
            #storing current position 
            current_position = np.array([position[0,i],position[1,i]])

            
            #calculating magnitude of positions
            mag = magnitude(current_position[0],current_position[1])
            
            #checking if in the same cluster, if so adding to cluster count
            if(mag<0.2*R and mag!=0):
                cluster_count = cluster_count+1
            
            #calculating the force
            numerator = (G*(robot.mass**2))
            denominator = mag**2

            
            #calling function to account for division by zero, if there is a zero denominator, then the force component with zero is returned as zero
            force_change = division_check(numerator,denominator)
            
            #calculating the unit vector of the position
            distance_unit = unit(current_position)
            
            #if the magnitude is bigger than R, then a force is added to draw the robots together             
            if(mag>R):
                force_change_dir = distance_unit
            
            #if the magnitude is smaller than R, then a force is added to push the robots apart 
            elif(mag<R):
                force_change_dir = -distance_unit
            else:
                force_change_dir = np.array([0,0])
       
            #calculating new force change based on direction and magnitude
            force_delta = force_change_dir*force_change

            #constraining force to the maximum
            if(force_delta[0] > max_force):
                force_delta[0] = max_force
            elif(force_delta[0]<-max_force):
                force_delta[0] = -max_force
            if(force_delta[1] > max_force):
                force_delta[1] = max_force
            elif(force_delta[1]<-max_force):
                force_delta[1] = -max_force
                

            #calculating new force
            force = force+force_delta
        cluster_total = cluster_total+cluster_count
        
        avg_min_distance = minimum_distance/len(swarm)


        #calculating the change in velocity
        delta_vx = force[0]*timestep/robot.mass
        delta_vy = force[1]*timestep/robot.mass

        #calculating the new velocity
        velocity_x = robot.x_velocity + delta_vx
        velocity_y = robot.y_velocity + delta_vy
        

        
        #keeping the velocity within the maximum velocity of the robot
        if(velocity_x>robot.max_velocity):
            velocity_x = robot.max_velocity
        if(velocity_y>robot.max_velocity):
            velocity_y = robot.max_velocity
        if(velocity_x<-robot.max_velocity):
            velocity_x = -robot.max_velocity
        if(velocity_y<-robot.max_velocity):
            velocity_y = -robot.max_velocity
            
        #updating the robots velocity
        robot.update_velocity(velocity_x,velocity_y)    

        #calculating the change in position for x and y
        x_change = velocity_x*timestep
        y_change = velocity_y*timestep

        #calculating the new position
        x_updated = robot.x+x_change
        y_updated = robot.y+y_change
        
        #updating the position
        robot.update_position(x_updated,y_updated)
        count = count+1
    cluster_avg = cluster_total/robot_number
    cluster_average.append(cluster_avg)    
   
   
    min_neighbours.append(avg_min_distance)

In [16]:
#Function to calculate artificial gravity forces
def artificial_gravity(robot, length, position, max_force,R,G,constant):
    '''
    Calculates the artificial gravitational force on each robot

            Parameters:
                   Robot (object): the robot object that this calculation is for
                   Length (float): float number of robots
                   position (float): the positions of the robots
                   max_force (float): the maximum float limit on the force
                   R (float): the chosen R seperation float value
                   G (float): the chosen G constant float value
                   constant (float): the range multiplication float
            Returns:
                   force (float): a float of the final force on the robot

    '''
    force = np.array([0,0])
    force_change_dir = 0

    for i in range(length):

        #storing current position 
        current_position = np.array([position[0,i],position[1,i]])


        #calculating magnitude of positions
        mag = (magnitude(current_position[0],current_position[1]))*constant


        #calculating the force
        numerator = (G*(robot.mass**2))
        denominator = mag**2


        #calling function to account for division by zero, if there is a zero denominator, then the force component with zero is returned as zero
        force_change = division_check(numerator,denominator)

        #calculating the unit vector of the position
        distance_unit = unit(current_position)

        #if the magnitude is bigger than R, then a force is added to draw the robots together             
        if(mag>R):
            force_change_dir = distance_unit

        #if the magnitude is smaller than R, then a force is added to push the robots apart 
        elif(mag<R):
            force_change_dir = -distance_unit
        else:
            force_change_dir = np.array([0,0])

        #calculating new force change based on direction and magnitude
        force_delta = force_change_dir*force_change

        #constraining force to the maximum
        if(force_delta[0] > max_force):
            force_delta[0] = max_force
        elif(force_delta[0]<-max_force):
            force_delta[0] = -max_force
        if(force_delta[1] > max_force):
            force_delta[1] = max_force
        elif(force_delta[1]<-max_force):
            force_delta[1] = -max_force
        force = force+force_delta
    return force
    

In [17]:
#Function to calculate the G transition parameter
def G_transition(max_force,R,power):
    '''
    Calculates the transition value between liquid and solid states for the G parameter

            Parameters:
                   max_force (float): the maximum float limit on the force
                   R (float): the chosen R seperation float value
                   power (float): the chosen power for the artificial gravity equation
                   
            Returns:
                   G_transition (float): the G transition value for the current swarm setup

    '''
    G_transition = (0.9*max_force*(R**power))/(2*math.sqrt(3))
    return G_transition

In [18]:
print(G_transition(1,200,2))

10392.304845413264


In [19]:
#Function to calculate distance between a robot and the swarm
def dist(robot, x,y):
    '''
    Calculates the distances between a robot and the swarm

            Parameters:
                   Robot (object): the robot object that this calculation is for
                   x (float): array of robot x positions
                   y (float): array of robot y positions
                   
            Returns:
                   x_dist (float): an array of the x distances from the current robot to the robot in the swarm
                   y_dist (float): an array of the y distances from the current robot to the robot in the swarm

    '''
    #finding the distance between the current robot and the other robots
    x_dist = x-robot.x
    y_dist = y-robot.y
    return x_dist,y_dist

In [20]:
#Function to split swarm into different types
def split(swarm):
    '''
    Returns arrays of the swarms split into their respective types

            Parameters:
                   swarm (object) : array of robot objects in the swarm
                   
            Returns:
                   type_0 (float): an array of the x distances from the current robot to the robot in the swarm
                   y_dist (float): an array of the y distances from the current robot to the robot in the swarm

    '''
    type_0 = []
    type_1 = []
    for robot in swarm:
        if(robot.identifier == 0):
            type_0.append(robot)
        if(robot.identifier == 1):
            type_1.append(robot)

            
    return type_0,type_1
    

In [21]:
#Function to create random walk
def random_walk(swarm):
        """
        Updates the movement of the robot by checking if the current heading needs to be reset, and then updating the position           according to the header

            Parameters:
                   Swarm (list): a list of robot objects
            
            Returns:
                   None
        """
        weighting = 20
        deviation = 5
    
        for robot in swarm:
            if robot.check_walk():
                walk_time = int(np.random.normal(weighting,deviation,1))
                heading = random.uniform(0,2*math.pi)
                robot.walk_reset(walk_time,heading)
            x_change = math.cos(robot.heading)*robot.x_velocity
            y_change = math.sin(robot.heading)*robot.y_velocity
            x_updated = robot.x + x_change
            y_updated = robot.y + y_change
            #self.bounce(x_updated,y_updated,x_change,y_change,60000)
            robot.update_position(x_updated,y_updated)
            robot.update_timer()

In [22]:
#Function to check area coverage
def density_measure(robot_number,block_size,total_area):
    '''
    Returns density measure of the swarm

            Parameters:
                    robot_number (int): number of robots in the swarm
                    block_size(float): square area of blocks desired
                    total_area(float): square area of designated area
            Returns:
                   density (float): density of robot per given block size
    '''
    
    block_number = ((total_area*1000)**2)/block_size
    print(block_number)
    density = robot_number/block_number
    print("Density: ", density)
    return density
    

In [23]:
#Function to return performance based on density and average number of clusters
def fitness_function(density_measure,cluster_average,com_range,min_neighbours):
    '''
    Returns performance measure that will ensure the cluster average metric is met, along with the density coverage for interception

            Parameters:
                    density_measure (float): number of robots per given area
                    cluster_average (float): number of robot clusters within a certain range average over number of robots
                    com_range (float): set communication range
                    min_neigbours (float): the performance metric of the minimum neighbour distance which is averaged over the whole swarm
            Returns:
                   performance (float): performance metric which gives a performance metric useful for moving targets where the sensor range isn't clear
    '''
    ratio = min_neighbours/com_range
    if(ratio>1):
        perform = 0
    else:
        perform = (density_measure/(cluster_average**10))*ratio
    return perform*10000

In [24]:

G_array = [1000,1500,2000,2500,3000,3500,4000,4500,5000,5500,6000,6500,7000,7500,8000,8500,9000,9500,10000]
#power_array = [2,3,4,5]
#R_array = [10,50,100,125,150,200,250]
#multiply_array = [1,1.1,1.2,1.3,1.4,1.5,2]
robot_number_array = [10,25,50,100]

#setting physics constants
power = 2
R = 150
max_force = 1
multiply = 1.5

#setting timestep parameter
time = 1000

current_directory = "Experiments/Parameter Sweep/"
try:
    os.mkdir(current_directory)
except OSError:
    print ("Failure: Directory creation of %s failed" % current_directory)
else:
    print ("Success: Directory creation of %s succeeded" % current_directory)


values = []
for robot in robot_number_array:

    robot_path = (current_directory + str(robot) + " Robots/")
    try:
        os.mkdir(robot_path)
    except OSError:
        print ("Failure: Directory creation of %s failed" % robot_path)
    else:
        print ("Success: Directory creation of %s succeeded" % robot_path)
    image = (current_directory + str(robot) + " Robots/Images/")
    try:
        os.mkdir(image)
    except OSError:
        print ("Failure: Directory creation of %s failed" % image)
    else:
        print ("Success: Directory creation of %s succeeded" % image)
    G_state = G_transition(max_force,R,power)
    #G_array = (np.arange(G_state*0.8, G_state*1.2, 0.02*G_state))
    
    for i in range(len(G_array)):
        random_seed = os.urandom(8)
        random.seed(random_seed)
        swarm = initialise(robot, side)
        store_robots = np.zeros((2,robot,time))
        min_neighbours = []
        cluster_average = []
        density = density_measure(robot,100,side)
        for j in range(time):
            #updating robots and storing
            store_robots[:,:,j] = positions(swarm)
            physics_walk(swarm,G_array[i],power,R,max_force,multiply,min_neighbours,cluster_average)
    
  
        #Experiment number
        code = "G" + str(np.around(G_array[i]))
        print(code)
        robots = (current_directory + str(robot) + " Robots/")
        path = (current_directory + str(robot) + " Robots/" + code + "/")
        try:
            os.mkdir(path)
        except OSError:
            print ("Failure: Directory creation of %s failed" % path)
        else:
            print ("Success: Directory creation of %s succeeded" % path)


        np.save(path + "Robots.npy",store_robots)
        
        values.append([len(swarm),G_array[i],power,R,multiply,max_force,robot_speed,min_neighbours[time-1],cluster_average[time-1],density,random_seed, G_state])
        
        data = {'Values':[len(swarm),G_array[i],power,R,multiply,max_force,robot_speed,min_neighbours,cluster_average,density,random_seed, G_state]}
        titles =['Number of Robots',
                                        'G', 
                                        'Power', 
                                        'Communication Range', 
                                        'Multiplier',
                                        'Max Force',
                                        'Max Speed',
                                        'Minimum Distance to Neighbours',
                                        'Average Cluster Size',
                                        'Density',
                                        'Seed',
                                        'G Transition']
        constants = pd.DataFrame(data, index = titles
                                       ) 

        constants.to_excel(path + "Constants.xlsx")

        graph_start_path = image + code + "- Graph_Beginning.png"
        table_start_path = image + "/" + code + "- Table_Beginning.png"
        
        graph_start = Processing_Functions.graph_figure(store_robots,0,code)
        graph_start.write_image(graph_start_path)
        table_start = Processing_Functions.table_figure(store_robots,0,constants)
        table_start.write_image(table_start_path)
        
        Processing_Functions.combine(graph_start_path,table_start_path)

        graph_end_path = image + code + "- Graph_End.png"
        table_end_path = image + "/" + code + "- Table_End.png"
        
        graph_end = Processing_Functions.graph_figure(store_robots,time-1,code)
        graph_end.write_image(image + "/" + code + "- Graph_End.png")
        table_end = Processing_Functions.table_figure(store_robots,time-1,constants)
        table_end.write_image(image + "/" + code + "- Table_End.png")

        Processing_Functions.combine(graph_end_path,table_end_path)
        
        performance = Processing_Functions.performance_graph(min_neighbours,code,"Time (s)","Minimum Average Neighbour Distance (m)")
        performance.write_image(image + "/" + code + "- Performance.png")
        
        performance = Processing_Functions.performance_graph(min_neighbours,code,"Time (s)","Minimum Average Neighbour Distance (m)")
        performance.write_image(image + "/" + code + "- Performance.png")
        
        cluster = Processing_Functions.performance_graph(cluster_average,code,"Time (s)","Average Cluster Size")
        cluster.write_image(image + "/" + code + "- Average Cluster Size.png")
        


df = pd.DataFrame(values, columns = titles)

df.to_excel(current_directory + "Tested Values.xlsx")

Success: Directory creation of Experiments/Parameter Sweep/ succeeded
Success: Directory creation of Experiments/Parameter Sweep/10 Robots/ succeeded
Success: Directory creation of Experiments/Parameter Sweep/10 Robots/Images/ succeeded
10000.0
Density:  0.001
G1000
Success: Directory creation of Experiments/Parameter Sweep/10 Robots/G1000/ succeeded
10000.0
Density:  0.001
G1500
Success: Directory creation of Experiments/Parameter Sweep/10 Robots/G1500/ succeeded
10000.0
Density:  0.001
G2000
Success: Directory creation of Experiments/Parameter Sweep/10 Robots/G2000/ succeeded
10000.0
Density:  0.001
G2500
Success: Directory creation of Experiments/Parameter Sweep/10 Robots/G2500/ succeeded
10000.0
Density:  0.001
G3000
Success: Directory creation of Experiments/Parameter Sweep/10 Robots/G3000/ succeeded
10000.0
Density:  0.001
G3500
Success: Directory creation of Experiments/Parameter Sweep/10 Robots/G3500/ succeeded
10000.0
Density:  0.001
G4000
Success: Directory creation of Experi

G4500
Success: Directory creation of Experiments/Parameter Sweep/100 Robots/G4500/ succeeded
10000.0
Density:  0.01
G5000
Success: Directory creation of Experiments/Parameter Sweep/100 Robots/G5000/ succeeded
10000.0
Density:  0.01
G5500
Success: Directory creation of Experiments/Parameter Sweep/100 Robots/G5500/ succeeded
10000.0
Density:  0.01
G6000
Success: Directory creation of Experiments/Parameter Sweep/100 Robots/G6000/ succeeded
10000.0
Density:  0.01
G6500
Success: Directory creation of Experiments/Parameter Sweep/100 Robots/G6500/ succeeded
10000.0
Density:  0.01
G7000
Success: Directory creation of Experiments/Parameter Sweep/100 Robots/G7000/ succeeded
10000.0
Density:  0.01
G7500
Success: Directory creation of Experiments/Parameter Sweep/100 Robots/G7500/ succeeded
10000.0
Density:  0.01
G8000
Success: Directory creation of Experiments/Parameter Sweep/100 Robots/G8000/ succeeded
10000.0
Density:  0.01
G8500
Success: Directory creation of Experiments/Parameter Sweep/100 Rob

In [25]:
for i in range(1,1000):
    print(i)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [26]:
print(dust_probability)

NameError: name 'dust_probability' is not defined